In [79]:
%load_ext autoreload
%autoreload 2
!pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/ayushalag/Documents/cosIW/cosIW_sp21


In [83]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('/Users/ayushalag/Documents/cosIW/cosIW_sp21/deluca/agents/__init__.py')
os.chdir('/Users/ayushalag/Documents/cosIW/cosIW_sp21/')
print(os.getcwd())
print(sys.path)
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))

/Users/ayushalag/Documents/cosIW/cosIW_sp21
['/Users/ayushalag/Documents/cosIW/cosIW_sp21/examples/agents', '/Users/ayushalag/anaconda3/lib/python37.zip', '/Users/ayushalag/anaconda3/lib/python3.7', '/Users/ayushalag/anaconda3/lib/python3.7/lib-dynload', '', '/Users/ayushalag/anaconda3/lib/python3.7/site-packages', '/Users/ayushalag/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/ayushalag/Documents/cosIW/deluca', '/Users/ayushalag/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/ayushalag/.ipython', '/Users/ayushalag/Documents/cosIW/cosIW_sp21/examples', '', '/Users/ayushalag/Documents/cosIW/cosIW_sp21/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents', '/Users/ayushalag/Documents/cosIW/', '/Users/ayushalag/Documents/cosIW', 

In [84]:
print(os.getcwd())
print(sys.executable)
from deluca.agents._gpc import GPC
from deluca.agents import GPC, Adaptive
from deluca.envs import LDS
import jax.numpy as jnp
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

/Users/ayushalag/Documents/cosIW/cosIW_sp21
/Users/ayushalag/anaconda3/bin/python


ModuleNotFoundError: No module named 'deluca.agents._gpc'

In [6]:
cummean = lambda x: np.cumsum(x)/(np.ones(T) + np.arange(T))

In [7]:
n, m = 2, 1

In [8]:
def get_errs(T, controller, A, B, noise):
    state = jnp.zeros((n, 1))
    errs = [0.]
    
    for i in tqdm(range(1, T)):
        
        try:
            action = controller(state, A, B)
        except:
            action = controller(state)
            
        if(noise=="normal"):
            state = A @ state + B @ action + np.random.normal(0, 0.2, size=(n,1)) # gaussian noise
        else:
            state = A @ state + B @ action + np.random.normal(0, 0.2, size=(n,1)) *(i%300 < 150) + 0.4 * jnp.sin(i) *(i%300>= 150) # add sine noise every 150 steps
        
        if(i % T//2 == 0): # switch system
            A,B = jnp.array([[1.,1.5], [0,1.]]), jnp.array([[0],[0.9]])
        
        errs.append(jnp.linalg.norm(state)+jnp.linalg.norm(action))
    
    return errs

In [9]:
T = 600

In [10]:
A,B = jnp.array([[1.,.5], [0,1.]]), jnp.array([[0],[1.2]])

ada = Adaptive(T, base_controller=GPC, A=A, B=B)
gpc = GPC(A, B)

gpc_errs = get_errs(T, gpc, A, B, "normal")
print("GPC incurs ", np.mean(gpc_errs), " loss under gaussian iid noise")

ada_errs = get_errs(T, ada, A, B, "normal")
print("AdaGPC incurs ", np.mean(ada_errs), " loss under gaussian iid noise")

plt.title("Instantenous losses under gaussian iid noise")
plt.plot(cummean(gpc_errs), "green", label = "GPC")
plt.plot(cummean(ada_errs), "blue", label = "AdaGPC")
plt.legend();

NameError: name 'jnp' is not defined

In [19]:
A,B = jnp.array([[1.,.5], [0,1.]]), jnp.array([[0],[1.2]])

ada = Adaptive(T, base_controller=GPC, A=A, B=B)
gpc = GPC(A, B)

gpc_errs = get_errs(T, gpc, A, B, "sine")
print("GPC incurs ", np.mean(gpc_errs), " loss under intermittent sine noise")

ada_errs = get_errs(T, ada, A, B, "sine")
print("AdaGPC incurs ", np.mean(ada_errs), " loss under intermittent sine noise")

plt.title("Instantenous losses under intermittent sine noise")
plt.plot(cummean(gpc_errs), "green", label = "GPC")
plt.plot(cummean(ada_errs), "blue", label = "AdaGPC")
plt.legend();

AttributeError: module 'jax' has no attribute 'core'